In [2]:
%reset -f

import pygame
import random
import time

# Inicialização do Pygame
pygame.init()

# Variáveis
simulation_speed = 0.1  # Velocidade inicial de 0.5 segundos

initial_fox_count = 5  # Quantidade inicial de raposas
initial_rabbit_count = 20  # Quantidade inicial de coelhos
initial_plant_count = 100  # Quantidade inicial de plantas

rabbit_lifespan = 20
fox_lifespan = 10

plant_generation_rate = 20  # Quantidade de plantas geradas por época


SCREEN_WIDTH, SCREEN_HEIGHT = 1920, 1200  # Definir o tamanho da tela

# Cores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (50, 255, 50)
BROWN = (70, 70, 70) 
BROWN_BRIGHT = (140, 140, 140)
RED = (255,50,50)
RED_BRIGHT = (255,120,120)

# Definir o tamanho da grid e o tamanho da célula
GRID_SIZE_METERS = 40  # 40 metros (lado)
CELL_SIZE = 20  # Cada célula será 20x20 pixels
GRID_SIZE = GRID_SIZE_METERS * CELL_SIZE  # Tamanho total da grid em pixels

# Calcular a posição para centralizar a grid
GRID_X = (SCREEN_WIDTH - GRID_SIZE) // 4
GRID_Y = (SCREEN_HEIGHT - GRID_SIZE) // 2

# Criar a janela
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Grid Centralizada")

# Classe Plant
class Plant:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Classe Rabbit
class Rabbit:
    def __init__(self, x, y, vision_radius=2):
        self.x = x
        self.y = y
        self.lifespan = rabbit_lifespan
        self.age = 0
        self.is_baby = True
        self.vision_radius = vision_radius  # Define o raio de visão

    def decide_move(self, grid):
        best_move = None
        for dx in range(-self.vision_radius, self.vision_radius + 1):
            for dy in range(-self.vision_radius, self.vision_radius + 1):
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < GRID_SIZE_METERS and 0 <= ny < GRID_SIZE_METERS:
                    cell = grid[nx][ny]
                    # Fugir de raposa se presente
                    if cell.has_fox():
                        best_move = (nx, ny)
                        return self.move_away(grid, best_move)  # Prioriza fugir de raposas
                    # Mover-se em direção à planta
                    elif cell.has_plant() and not cell.has_fox():
                        best_move = (nx, ny)

        # Se não há melhor movimento encontrado, move aleatoriamente
        if not best_move:
            self.random_move()
        else:
            self.move_to(best_move, grid)

    def move_to(self, position, grid):
        target_cell = grid[position[0]][position[1]]
        if target_cell.rabbit_count == 0:  # Check if the cell is empty
            self.x, self.y = position

    def random_move(self):
        # Movimento aleatório
        direction = random.choice(['up', 'down', 'left', 'right'])
        if direction == 'up' and self.y > 0:
            self.y -= 1
        elif direction == 'down' and self.y < GRID_SIZE_METERS - 1:
            self.y += 1
        elif direction == 'left' and self.x > 0:
            self.x -= 1
        elif direction == 'right' and self.x < GRID_SIZE_METERS - 1:
            self.x += 1

    def move_away(self, grid, fox_position):
        # Move o coelho para longe da raposa mais próxima
        direction_x = self.x - fox_position[0]
        direction_y = self.y - fox_position[1]
        new_x = self.x + (1 if direction_x > 0 else -1 if direction_x < 0 else 0)
        new_y = self.y + (1 if direction_y > 0 else -1 if direction_y < 0 else 0)
        if 0 <= new_x < GRID_SIZE_METERS and 0 <= new_y < GRID_SIZE_METERS:
            self.x, self.y = new_x, new_y

    def eat_plants_and_reproduce(self, grid, rabbits, plants_to_remove):
        cell = grid[self.x][self.y]
        if cell.has_plant():
            # Marcar a planta para ser removida
            plants_to_remove.append((self.x, self.y))
            self.reproduce(grid, rabbits)

    def reproduce(self, grid, rabbits):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        available_positions = [(self.x + dx, self.y + dy) for dx, dy in directions
                               if 0 <= self.x + dx < GRID_SIZE_METERS and 0 <= self.y + dy < GRID_SIZE_METERS]

        available_positions = [pos for pos in available_positions if not grid[pos[0]][pos[1]].has_rabbit()]

        if available_positions:
            new_x, new_y = random.choice(available_positions)
            new_rabbit = Rabbit(new_x, new_y)
            rabbits.append(new_rabbit)
            grid[new_x][new_y].add_entity(new_rabbit)

    def age_and_check_if_alive(self):
        self.age += 1
        return self.age < self.lifespan

class Fox:
    def __init__(self, x, y, vision_radius=1):
        self.x = x
        self.y = y
        self.lifespan = fox_lifespan
        self.age = 0
        self.is_baby = True
        self.vision_radius = vision_radius  # Define o raio de visão

    def decide_move(self, grid):
        best_move = None
        for dx in range(-self.vision_radius, self.vision_radius + 1):
            for dy in range(-self.vision_radius, self.vision_radius + 1):
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < GRID_SIZE_METERS and 0 <= ny < GRID_SIZE_METERS:
                    cell = grid[nx][ny]
                    # Move em direção ao coelho
                    if cell.has_rabbit():
                        best_move = (nx, ny)
                        break
        if best_move:
            self.move_to(best_move, grid)
        else:
            self.random_move()

    def move_to(self, position, grid):
        target_cell = grid[position[0]][position[1]]
        if target_cell.fox_count == 0:  # Check if the cell is empty
            self.x, self.y = position

    def random_move(self):
        # Movimento aleatório para explorar o espaço
        direction = random.choice(['up', 'down', 'left', 'right'])
        if direction == 'up' and self.y > 0:
            self.y -= 1
        elif direction == 'down' and self.y < GRID_SIZE_METERS - 1:
            self.y += 1
        elif direction == 'left' and self.x > 0:
            self.x -= 1
        elif direction == 'right' and self.x < GRID_SIZE_METERS - 1:
            self.x += 1

    def eat_rabbits_and_reproduce(self, grid, foxes, rabbits_to_remove):
        cell = grid[self.x][self.y]
        if cell.has_rabbit():
            # Marcar o coelho para ser removido
            rabbits_to_remove.append((self.x, self.y))
            self.reproduce(grid, foxes)

    def reproduce(self, grid, foxes):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        available_positions = [(self.x + dx, self.y + dy) for dx, dy in directions
            if 0 <= self.x + dx < GRID_SIZE_METERS and 0 <= self.y + dy < GRID_SIZE_METERS]

        # Checar se a posição está livre de raposas
        available_positions = [pos for pos in available_positions if not grid[pos[0]][pos[1]].has_fox()]

        if available_positions:
            new_x, new_y = random.choice(available_positions)
            new_fox = Fox(new_x, new_y)
            foxes.append(new_fox)
            grid[new_x][new_y].add_entity(new_fox)

    def age_and_check_if_alive(self):
        self.age += 1
        return self.age < self.lifespan


class GridCell:
    def __init__(self):
        self.entities = []
        self.fox_count = 0
        self.rabbit_count = 0
        self.plant_count = 0


    def add_entity(self, entity):
        self.entities.append(entity)
        if isinstance(entity, Rabbit):
            self.rabbit_count += 1
        elif isinstance(entity, Plant):
            self.plant_count += 1
        elif isinstance(entity, Fox):
            self.fox_count += 1

    def remove_entity(self, entity_type):
        self.entities = [entity for entity in self.entities if not isinstance(entity, entity_type)]
        if entity_type == Rabbit:
            self.rabbit_count -= 1
        elif entity_type == Plant:
            self.plant_count -= 1
        elif entity_type == Fox:
            self.fox_count -= 1

    def has_rabbit(self):
        return self.rabbit_count > 0

    def has_plant(self):
        return self.plant_count > 0

    def has_fox(self):
        return self.fox_count > 0

    def get_entity_type(self):
        # Return 3 for fox, 2 for rabbit, 1 for plant, 0 for empty
        if self.has_fox():
            return 3
        elif self.has_rabbit():
            return 2
        elif self.has_plant():
            return 1
        else:
            return 0

    def clear(self):
        self.entities.clear()
        self.rabbit_count = 0
        self.plant_count = 0
        self.fox_count = 0


    def debug_print(self):
        if self.has_rabbit():
            return "Rabbit"
        elif self.has_plant():
            return "Plant"
        else:
            return "Empty"

def draw_cell_numbers(grid):
    font = pygame.font.SysFont(None, 24)
    for row in range(GRID_SIZE_METERS):
        for col in range(GRID_SIZE_METERS):
            rect_x = GRID_X + col * CELL_SIZE
            rect_y = GRID_Y + row * CELL_SIZE
            cell = grid[row][col]

            # Get the entity type (2 for rabbit, 1 for plant, 0 for empty)
            number = cell.get_entity_type()

            # Draw the number in the cell
            text_surface = font.render(str(number), True, BLACK)
            screen.blit(text_surface, (rect_x + CELL_SIZE // 2 - text_surface.get_width() // 2, 
                                       rect_y + CELL_SIZE // 2 - text_surface.get_height() // 2))

def draw_grid(grid):
    for row in range(GRID_SIZE_METERS):
        for col in range(GRID_SIZE_METERS):
            rect = pygame.Rect(GRID_X + col * CELL_SIZE, GRID_Y + row * CELL_SIZE, CELL_SIZE, CELL_SIZE)

            # Define um valor padrão para a cor
            color = WHITE  # Cor padrão

            if grid[row][col].has_rabbit():
                for entity in grid[row][col].entities:
                    if isinstance(entity, Rabbit):
                        color = BROWN if not entity.is_baby else BROWN_BRIGHT
                        break
            elif grid[row][col].has_fox():
                for entity in grid[row][col].entities:
                    if isinstance(entity, Fox):
                        color = RED if not entity.is_baby else RED_BRIGHT
                        break
            elif grid[row][col].has_plant():
                color = GREEN
            
            pygame.draw.rect(screen, color, rect)
            pygame.draw.rect(screen, BLACK, rect, 1)

                             

def update_grid(grid, plants, rabbits, foxes):
    for row in grid:
        for cell in row:
            cell.clear()

    for plant in plants:
        grid[plant.x][plant.y].add_entity(plant)
    
    for rabbit in rabbits:
        grid[rabbit.x][rabbit.y].add_entity(rabbit)
    
    for fox in foxes:
        grid[fox.x][fox.y].add_entity(fox)

        

def draw_graph(rabbit_counts, plant_counts, fox_counts):
    # Configurações do gráfico
    max_count = max(max(rabbit_counts, default=0), max(plant_counts, default=0), max(fox_counts, default=0))  # Contagem máxima
    graph_width = 500
    graph_height = 300
    start_x = 1140
    start_y = 700

    # Desenhar fundo do gráfico
    pygame.draw.rect(screen, WHITE, (start_x, start_y, graph_width, graph_height))
    pygame.draw.rect(screen, BLACK, (start_x, start_y, graph_width, graph_height), 2)  # Borda do gráfico

    # Desenhar eixo Y
    for i in range(0, max_count + 1, max(1, max_count // 10)):  # Divide o eixo Y em partes
        y = start_y + graph_height - (graph_height * i / max_count)
        pygame.draw.line(screen, BLACK, (start_x, y), (start_x + graph_width, y), 1)  # Linhas horizontais
        font = pygame.font.Font(None, 20)
        text = font.render(str(i), True, BLACK)
        screen.blit(text, (start_x - 30, y - 10))

    # Desenhar eixo X
    for i in range(0, len(rabbit_counts), max(1, len(rabbit_counts) // 10)):  # Divide o eixo X em partes
        x = start_x + (i * graph_width // len(rabbit_counts))
        pygame.draw.line(screen, BLACK, (x, start_y), (x, start_y + graph_height), 1)  # Linhas verticais

    # Desenhar linhas do gráfico
    if len(plant_counts) > 1:
        for i in range(len(plant_counts) - 1):
            pygame.draw.line(screen, GREEN,
                             (start_x + (i * graph_width // len(plant_counts)), start_y + graph_height - (graph_height * plant_counts[i] / max_count)),
                             (start_x + ((i + 1) * graph_width // len(plant_counts)), start_y + graph_height - (graph_height * plant_counts[i + 1] / max_count)), 2)  # Linha marrom para plantas
    if len(rabbit_counts) > 1:
        for i in range(len(rabbit_counts) - 1):
            pygame.draw.line(screen, BROWN, 
                             (start_x + (i * graph_width // len(rabbit_counts)), start_y + graph_height - (graph_height * rabbit_counts[i] / max_count)),
                             (start_x + ((i + 1) * graph_width // len(rabbit_counts)), start_y + graph_height - (graph_height * rabbit_counts[i + 1] / max_count)), 2)  # Linha verde para coelhos
    if len(fox_counts) > 1:
        for i in range(len(fox_counts) - 1):
            pygame.draw.line(screen, RED, 
                             (start_x + (i * graph_width // len(fox_counts)), start_y + graph_height - (graph_height * fox_counts[i] / max_count)),
                             (start_x + ((i + 1) * graph_width // len(fox_counts)), start_y + graph_height - (graph_height * fox_counts[i + 1] / max_count)), 2)  # Linha verde para coelhos                    
    
    # Adicionar legendas
    font = pygame.font.Font(None, 24)
    label_individuals = font.render("Indivíduos", True, BLACK)
    label_generations = font.render("Gerações", True, BLACK)

    # Desenhar "Indivíduos" verticalmente
    rotated_label_individuals = pygame.transform.rotate(label_individuals, 90)  # Rotaciona 90 graus
    screen.blit(rotated_label_individuals, (start_x - 50, start_y + graph_height // 2 - rotated_label_individuals.get_width() // 2))

    # Desenhar "Gerações" horizontalmente
    screen.blit(label_generations, (start_x + graph_width // 2 - 40, start_y + graph_height + 5))

def generate_plants(num_plants, grid):
    plants = []
    available_positions = [(x, y) for x in range(GRID_SIZE_METERS) for y in range(GRID_SIZE_METERS)
                           if not grid[x][y].entities]

    if available_positions:
        num_plants = min(num_plants, len(available_positions))

        for _ in range(num_plants):
            x, y = random.choice(available_positions)
            plant = Plant(x, y)
            plants.append(plant)
            grid[x][y].add_entity(plant)
            available_positions.remove((x, y))
    return plants

def draw_button(text, x, y, width, height, active):
    color = BLACK if active else WHITE
    pygame.draw.rect(screen, color, (x, y, width, height), 0)
    pygame.draw.rect(screen, BLACK, (x, y, width, height), 3)
    font = pygame.font.SysFont(None, 48)
    text_surface = font.render(text, True, WHITE)
    screen.blit(text_surface, (x + (width - text_surface.get_width()) // 2, y + (height - text_surface.get_height()) // 2))


def draw_legend():
    font = pygame.font.SysFont(None, 36)
    legend_x = GRID_X + GRID_SIZE + 20
    legend_y = GRID_Y

    legend_text = ["Legend:", "0: Empty", "1: Plant", "2: Rabbit", "3: Fox"]
    for i, text in enumerate(legend_text):
        text_surface = font.render(text, True, BLACK)
        screen.blit(text_surface, (legend_x, legend_y + i * 30))

def main():
    clock = pygame.time.Clock()
    running = True
    show_grid = False
    plants = []  # Lista de plantas
    rabbits = []
    foxes = []

    # Listas para armazenar a quantidade de indivíduos ao longo do tempo
    plant_counts = []
    rabbit_counts = []
    fox_counts = []

    grid = [[GridCell() for _ in range(GRID_SIZE_METERS)] for _ in range(GRID_SIZE_METERS)]

    button_rect = pygame.Rect(SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 - 50, 200, 100)  # Start button no centro
    kill_plants_button_rect = pygame.Rect(1450, 200, 200, 50)  # Botão "Kill Plants"
    kill_rabbits_button_rect = pygame.Rect(1450, 260, 200, 50)  # Botão "Kill Rabbits"
    kill_fox_button_rect = pygame.Rect(1450, 320, 200, 50)  # Botão "Kill Fox"

    simulation_started = False
    last_plant_time = time.time()
    last_rabbit_move = time.time()
    last_fox_move = time.time()

    while running:
        screen.fill(WHITE)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                if not simulation_started and button_rect.collidepoint(event.pos):
                    show_grid = True
                    simulation_started = True
                    plants = generate_plants(initial_plant_count, grid)

                    for _ in range(initial_rabbit_count):
                        while True:
                            x, y = random.randint(0, GRID_SIZE_METERS - 1), random.randint(0, GRID_SIZE_METERS - 1)
                            if not grid[x][y].has_rabbit():
                                rabbit = Rabbit(x, y)
                                rabbits.append(rabbit)
                                grid[x][y].add_entity(rabbit)
                                break

                    for _ in range(initial_fox_count):
                        x, y = random.randint(0, GRID_SIZE_METERS - 1), random.randint(0, GRID_SIZE_METERS - 1)
                        new_fox = Fox(x, y)
                        foxes.append(new_fox)
                        grid[x][y].add_entity(new_fox)

                if kill_plants_button_rect.collidepoint(event.pos):
                    for plant in plants[:]:
                        grid[plant.x][plant.y].remove_entity(Plant)
                    plants.clear()

                if kill_rabbits_button_rect.collidepoint(event.pos):
                    for rabbit in rabbits[:]:
                        grid[rabbit.x][rabbit.y].remove_entity(Rabbit)
                    rabbits.clear()

                if kill_fox_button_rect.collidepoint(event.pos):
                    for fox in foxes[:]:
                        grid[fox.x][fox.y].remove_entity(Fox)
                    foxes.clear()

        if show_grid:
            update_grid(grid, plants, rabbits, foxes)
            rabbit_counts.append(len(rabbits))
            plant_counts.append(len(plants))
            fox_counts.append(len(foxes))
            draw_graph(rabbit_counts, plant_counts, fox_counts)

            if time.time() - last_plant_time >= simulation_speed:
                new_plants = generate_plants(plant_generation_rate, grid)
                plants.extend(new_plants)
                last_plant_time = time.time()

            if time.time() - last_rabbit_move >= simulation_speed:
                plants_to_remove = []
                for rabbit in rabbits[:]:
                    rabbit.decide_move(grid)  # Decide e executa o movimento com base no grid
                    rabbit.eat_plants_and_reproduce(grid, rabbits, plants_to_remove)
                    rabbit.is_baby = rabbit.age < 1
                    if not rabbit.age_and_check_if_alive():
                        rabbits.remove(rabbit)

                for x, y in plants_to_remove:
                    grid[x][y].remove_entity(Plant)
                    plants = [plant for plant in plants if (plant.x, plant.y) != (x, y)]

                last_rabbit_move = time.time()

            if time.time() - last_fox_move >= simulation_speed:
                rabbits_to_remove = []
                for fox in foxes[:]:
                    fox.decide_move(grid)  # Decide e executa o movimento com base no grid
                    fox.eat_rabbits_and_reproduce(grid, foxes, rabbits_to_remove)
                    fox.is_baby = fox.age < 1

                    if not fox.age_and_check_if_alive():
                        foxes.remove(fox)

                for x, y in rabbits_to_remove:
                    grid[x][y].remove_entity(Rabbit)
                    rabbits = [rabbit for rabbit in rabbits if (rabbit.x, rabbit.y) != (x, y)]

                last_fox_move = time.time()  # Atualizado para "last_fox_move"

            draw_grid(grid)
            draw_cell_numbers(grid)
            draw_legend()

            draw_button("Kill Rabbits", kill_rabbits_button_rect.x, kill_rabbits_button_rect.y,
                        kill_rabbits_button_rect.width, kill_rabbits_button_rect.height, True)
            draw_button("Kill Plants", kill_plants_button_rect.x, kill_plants_button_rect.y,
                        kill_plants_button_rect.width, kill_plants_button_rect.height, True)
            draw_button("Kill Fox", kill_fox_button_rect.x, kill_fox_button_rect.y,
                        kill_fox_button_rect.width, kill_fox_button_rect.height, True)
        else:
            draw_button("Start", button_rect.x, button_rect.y, button_rect.width, button_rect.height, True)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

main()
